# Imputation notebook 

In [129]:
import pandas as pd             
import numpy as np

import seaborn as sns
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   

import matplotlib.pyplot as plt
%matplotlib inline

import warnings 
warnings.filterwarnings("ignore")

In [130]:
train_data = pd.read_csv('./data/train.csv')

In [131]:
train_data.shape

(523021, 36)

#### Shallow copy

In [132]:
train_data_imputed=train_data.copy()

## Finding the variables that contain NANs (will need to be imputed)

In [133]:
#Isnull() checks for nans
#any() returns true if there is any true in an array
#values() then converts everything into a single array
areasWithNansMask = train_data_imputed.isnull().any().values

#This is how we access column titles
areasWithNans = train_data_imputed.columns.values

#using boolean indexing here
areasWithNans = areasWithNans[areasWithNansMask]
print("Variable that will need to be imputed:\n", areasWithNans)

Variable that will need to be imputed:
 ['CloudCover' 'Events' 'Max_Gust_SpeedKm_h' 'Max_VisibilityKm'
 'Mean_VisibilityKm' 'Min_VisibilitykM']


# Counting the nan and non nan values

In [134]:
#Isnull() checks for nans
#any() returns true if there is any true in an array
#values() then converts everything into a single array
areasWithNansMask = train_data.isnull().any().values

#This is how we access column titles
areasWithNans = train_data.columns.values

#using boolean indexing here
areasWithNans = areasWithNans[areasWithNansMask]
print("Variable that will need to be imputed", areasWithNans)

Variable that will need to be imputed ['CloudCover' 'Events' 'Max_Gust_SpeedKm_h' 'Max_VisibilityKm'
 'Mean_VisibilityKm' 'Min_VisibilitykM']


### Basic stats 

In [135]:
samples=len(train_data)
print("The variables and the count of null and not null \n")
#After using the isnull function, replace all instances of false to string "not null" and true instances to "Null"
#This makes it much more readable when using the value_counts function
cloudCoverPrintList = train_data['CloudCover'].isnull().replace(False, "Not Null").replace(True, "Null").value_counts()
print("CloudCover Null values:\n",cloudCoverPrintList)
nCloudCover=len(train_data[train_data["CloudCover"].isnull()])
print("Null values percentage:  %.2f" % (nCloudCover/samples*100),"%", "\n")

eventsPrintList = train_data['Events'].isnull().replace(False, "Not Null").replace(True, "Null").value_counts()
print("Events Null values:\n",eventsPrintList)
nEvents=len(train_data[train_data["Events"].isnull()])
print("Null values percentage:  %.2f" % (nEvents/samples*100),"%", "\n")


maxGustPrintList = train_data['Max_Gust_SpeedKm_h'].isnull().replace(False, "Not Null").replace(True, "Null").value_counts()
print("Max_Gust_SpeedKm_h Null values:\n",maxGustPrintList)
nMax_Gust_SpeedKm_h=len(train_data[train_data["Max_Gust_SpeedKm_h"].isnull()])
print("Null values percentage:  %.2f" % (nMax_Gust_SpeedKm_h/samples*100),"%", "\n")


maxVisPrintList=train_data['Max_VisibilityKm'].isnull().replace(False, "Not Null").replace(True, "Null").value_counts()
print("Max_VisibilityKm Null values:\n",maxVisPrintList)
nMax_VisibilityKm=len(train_data[train_data["Max_VisibilityKm"].isnull()])
print("Null values percentage:  %.2f" % (nMax_VisibilityKm/samples*100),"%", "\n")


meanVisPrintList=train_data['Mean_VisibilityKm'].isnull().replace(False, "Not Null").replace(True, "Null").value_counts()
print("Mean_VisibilityKm Null values:\n", meanVisPrintList)
nMean_VisibilityKm=len(train_data[train_data["Mean_VisibilityKm"].isnull()])
print("Null values percentage:  %.2f" % (nMean_VisibilityKm/samples*100),"%", "\n")


minVisPrintList=train_data['Min_VisibilitykM'].isnull().replace(False, "Not Null").replace(True, "Null").value_counts()
print("Min_VisibilitykM Null values:\n",minVisPrintList , "\n")
nminVisPrintList=len(train_data[train_data["Min_VisibilitykM"].isnull()])
print("Null values percentage:  %.2f" % (nminVisPrintList/samples*100),"%", "\n")



The variables and the count of null and not null 

CloudCover Null values:
 Not Null    481840
Null         41181
Name: CloudCover, dtype: int64
Null values percentage:  7.87 % 

Events Null values:
 Not Null    398923
Null        124098
Name: Events, dtype: int64
Null values percentage:  23.73 % 

Max_Gust_SpeedKm_h Null values:
 Null        409947
Not Null    113074
Name: Max_Gust_SpeedKm_h, dtype: int64
Null values percentage:  78.38 % 

Max_VisibilityKm Null values:
 Not Null    511683
Null         11338
Name: Max_VisibilityKm, dtype: int64
Null values percentage:  2.17 % 

Mean_VisibilityKm Null values:
 Not Null    511683
Null         11338
Name: Mean_VisibilityKm, dtype: int64
Null values percentage:  2.17 % 

Min_VisibilitykM Null values:
 Not Null    511683
Null         11338
Name: Min_VisibilitykM, dtype: int64 

Null values percentage:  2.17 % 



## ***Max_Gust_SpeedKm_h*** column
### Imputation dropping rows
We start with dropping the Max_Gust_SpeedKm_h column since it has too many missing values >78%  (as seen above in the variable count)

In [136]:
#Drop Max_Gust regardless because it has too many missing values
train_data_imputed = train_data_imputed.drop(["Max_Gust_SpeedKm_h"], axis=1)
#print("Without Max gust speed now: \n",new.head())
print("Datase without Max gust speed now: \n")
print(train_data_imputed.shape)

Datase without Max gust speed now: 

(523021, 35)


## ***Max_Gust_SpeedKm_h*** column
### Imputation splitting events and adding *Clear* when NAN
Here we fix the events column, 
converting nans to some other value

#### Filling Nan Values with clear

In [137]:
print("droping rows with nans after fixing the events column")
train_data_imputed['Events'] = train_data_imputed['Events'].fillna("Clear")
print("Without Events now: \n", train_data_imputed["Events"].isnull().sum())


droping rows with nans after fixing the events column
Without Events now: 
 0


#### Transform values into dummies - copy pasted 

In [138]:
print("Number of attributes before making the dummies: "+str(train_data_imputed.shape[1]))
dummies_event=train_data_imputed['Events'].str.get_dummies(sep='-')
train_data_imputed=train_data_imputed.drop('Events',axis=1)
train_data_imputed=pd.concat([train_data_imputed,dummies_event],axis=1)
print("Number of attributes after making the dummies: "+str(train_data_imputed.shape[1]))

Number of attributes before making the dummies: 35
Number of attributes after making the dummies: 40


## ***CloudCover*** colum
### Nan values to 0.0 
Set *CloudCover* to 0 if NaN,considering that everytime CloudCover is NaN (but 56 times), Precipitationmm is always 0


In [139]:
train_data_imputed['CloudCover'] = train_data_imputed['CloudCover'].fillna(0.0)

## ***VisibilitykM*** colum
### Drop nan values
I was wondering if it is possible to impute with the mean of that store
Or even if the day before or after make sense 

In [140]:
train_data_imputed =train_data_imputed.dropna(subset=['Max_VisibilityKm','Min_VisibilitykM','Mean_VisibilityKm']);

In [141]:
unknown_per_columns=train_data_imputed.isnull().sum()
unknown_per_columns

StoreID                       0
Date                          0
IsHoliday                     0
IsOpen                        0
HasPromotions                 0
StoreType                     0
AssortmentType                0
NearestCompetitor             0
Region                        0
NumberOfCustomers             0
NumberOfSales                 0
Region_AreaKM2                0
Region_GDP                    0
Region_PopulationK            0
CloudCover                    0
Max_Dew_PointC                0
Max_Humidity                  0
Max_Sea_Level_PressurehPa     0
Max_TemperatureC              0
Max_VisibilityKm              0
Max_Wind_SpeedKm_h            0
Mean_Dew_PointC               0
Mean_Humidity                 0
Mean_Sea_Level_PressurehPa    0
Mean_TemperatureC             0
Mean_VisibilityKm             0
Mean_Wind_SpeedKm_h           0
Min_Dew_PointC                0
Min_Humidity                  0
Min_Sea_Level_PressurehPa     0
Min_TemperatureC              0
Min_Visi

## Transform categorical Values into dummies 
### ***StoreType*** 

In [142]:
dummies_event=train_data_imputed['StoreType'].str.get_dummies()
train_data_imputed=train_data_imputed.drop('StoreType',axis=1)
train_data_imputed=pd.concat([train_data_imputed,dummies_event],axis=1)


### ***Assortment*** 

In [144]:
dummies_event=train_data_imputed['AssortmentType'].str.get_dummies()
train_data_imputed=train_data_imputed.drop('AssortmentType',axis=1)
train_data_imputed=pd.concat([train_data_imputed,dummies_event],axis=1)

## Parse Date 

In [145]:
train_data_imputed['Date']=pd.to_datetime(train_data_imputed['Date'])

## Replace spaces 

In [148]:
train_data_imputed.columns=train_data_imputed.columns.str.replace('\s+', '_')

# Save imputed dataset 

In [150]:
train_data_imputed.to_csv('./data/dataset_imputed.csv', index=False)